# **1. EXTRACT**

ETL stands for extract, transform and load. For developing ETL pipelines, we need to extract data from various sources, transform it into a usable format, and load it into a data warehouse or database for analysis.

Task is to develop a data ETL pipeline for the Fashion MNIST dataset. The dataset contains 70,000 grayscale images of size 28×28 pixels, categorized into ten clothing categories like t-shirts, dresses, sneakers, etc. The pipeline should extract the dataset from the data source, and perform the necessary transformations such as scaling, normalization and feature engineering.

Load the preprocessed data into an SQLite database for storage and easy retrieval.

To develop a Data ETL pipeline using Python, the first step is to collect data from a data source. Let’s use the Fashion-MNIST dataset provided by the Keras library 

In [ ]:
import tensorflow.keras as keras
(xtrain, ytrain), (xtest, ytest) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


# **2. TRANSFORM**

Now let’s clean and transform the data. Here we will normalize the pixel values to be between 0 and 1 and reshape the data into a 4D tensor

**WHY?**

We need to normalize the pixel values to be between 0 and 1 in order to ensure that all the input features (i.e., the pixel values) have the same scale. This is important because machine learning algorithms often work better when the input features are on the same scale, as this helps to prevent some features from dominating others. For example, if one pixel has a range of 0-255 and another pixel has a range of 0-1, the first pixel may end up dominating the second pixel in the calculations. By normalizing the pixel values to be between 0 and 1, we ensure that all the input features are on the same scale and that no one feature dominates the others.

Reshaping the data into a 4D tensor is also important because it allows us to represent the images as matrices or arrays with dimensions (samples, height, width, channels). The first dimension represents the number of images in our dataset, the second and third dimensions represent the height and width of each image, respectively, and the fourth dimension represents the number of channels (e.g., 1 for grayscale images and 3 for RGB images). By representing the images in this way, we can use convolutional neural networks (CNNs) to extract features from the images and classify them accurately.

In [ ]:
import numpy as np

xtrain = xtrain.astype('float32') / 255
xtest = xtest.astype('float32') / 255

xtrain = np.reshape(xtrain, (xtrain.shape[0], 28, 28, 1))
xtest = np.reshape(xtest, (xtest.shape[0], 28, 28, 1))

print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


# **3. LOAD**

Now let’s load the data into a database. We can use SQLite to create a database and load the data into it

In [ ]:
import sqlite3

conn = sqlite3.connect('fashion_mnist.db')

conn.execute('''CREATE TABLE IF NOT EXISTS images
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
             image BLOB NOT NULL,
             label INTEGER NOT NULL);''')

for i in range(xtrain.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtrain[i]), ytrain[i]])

conn.commit()

for i in range(xtest.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtest[i]), ytest[i]])

conn.commit()

conn.close()

In the above code:

1. The first line imports a library called sqlite3, which allows us to work with SQLite databases in Python.
2. We then create a connection to the database.
3. Next, we create a table in the database called “images”.
4. We use a loop to loop through each image in the training data and insert it into the “images” table (along with the labels).
5. We use the commit() method to save the changes we made to the database.
6. We then use another loop to loop through each image in the test data and insert it into the “images” table (along with the labels).
7. We use the commit() method again to save the changes we made to the database.
8. Finally, we close the connection to the database.

So this is how we can create a Data ETL pipeline using Python. Our ETL pipeline takes the Fashion MNIST dataset and stores it in an SQLite database so that we can easily access and manipulate the data later.

Reading the data you stored on the SQLite database

In [ ]:
import sqlite3
conn = sqlite3.connect('fashion_mnist.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM images')
rows = cursor.fetchall()

import pandas as pd
data = pd.read_sql_query('SELECT * FROM images', conn)

Data ETL is a process where data is extracted from a place, then the data is transformed in some way, and then data is loaded into a database. So ETL stands for Extracting, Transforming, and Loading the data. I hope you liked this article on developing a Data ETL pipeline using Python.